# Pre-annotation with Pixano [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pixano/pixano/blob/main/notebooks/models/pre_annotation.ipynb)

This notebook allows you to preannotate a whole Pixano dataset using a deep learning model.

You can then browse the inferences and compare them to ground truths in the Pixano app.

## 1. Setting up

### Install dependencies

This notebook requires installing `pixano` as well the models from the complementary [`pixano-inference`](https://github.com/pixano/pixano-inference) module.

If you are running this notebook on your computer, we strongly recommend creating a virtual environment for using Pixano like so:

```shell
conda create -n pixano_env python=3.10
conda activate pixano_env
```

```shell
pip install pixano
pip install pixano-inference@git+https://github.com/pixano/pixano-inference
```

If you are running this notebook in Google Colab, run the cell below to install `pixano` and `pixano-inference`.

In [ ]:
try:
    import google.colab

    ENV = "colab"
    %pip install pixano
    %pip install pixano-inference@git+https://github.com/pixano/pixano-inference
except:
    ENV = "jupyter"

### Load dependencies

In [ ]:
from pathlib import Path

from pixano_inference import pytorch, github, tensorflow

from pixano.app import App

### Download checkpoints and configs
- GroundingDINO:
    - SwinT checkpoint: [groundingdino_swint_ogc.pth](https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth)
    - SwinT config: [GroundingDINO_SwinT_OGC.py](https://raw.githubusercontent.com/IDEA-Research/GroundingDINO/main/groundingdino/config/GroundingDINO_SwinT_OGC.py)
    - SwinB checkpoint: [groundingdino_swinb_cogcoor.pth](https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha2/groundingdino_swinb_cogcoor.pth)
    - SwinB config: [GroundingDINO_SwinB_cfg.py](https://raw.githubusercontent.com/IDEA-Research/GroundingDINO/main/groundingdino/config/GroundingDINO_SwinB_cfg.py)
- Segment Anything Model:
    - ViT-H checkpoint: [sam_vit_h_4b8939.pth](https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth)
    - ViT-L checkpoint: [sam_vit_l_0b3195.pth](https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth)
    - ViT-B checkpoint: [sam_vit_b_01ec64.pth](https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth)
- MobileSAM:
    - ViT-T checkpoint: [mobile_sam.pt](https://github.com/ChaoningZhang/MobileSAM/raw/master/weights/mobile_sam.pt)

## 2. Genererating inferences
With the Pixano Inference module, we provide some useful models from the PyTorch and TensorFlow model hubs, as well as the Segment Anything Model (SAM) from Meta.

You can take a look at how they are implemented to add your own, or reach out to us if you think Pixano Inference could benefit from it, and we will try to add it in a future version. 

### Select a model
- Object Detection (COCO labels):
```python
model = pytorch.YOLOv5(size="m")
model = tensorflow.FasterRCNN()
model = tensorflow.EfficientDet()
```
- Instance Segmentation (COCO labels):
```python
model = pytorch.MaskRCNNv2()
```
- Semantic Segmentation (VOC labels):
```python
model = pytorch.DeepLabV3()
```
- Semantic Segmentation (text prompts):
```python
model = github.GroundingDINO(
    checkpoint_path=Path("my_datasets/models/groundingdino_swint_ogc.pth"),
    config_path=Path("my_datasets/models/GroundingDINO_SwinT_OGC.py")
)
```
- Segment Anything (no labels):
```python
model = github.SAM(
    checkpoint_path=Path("my_datasets/models/sam_vit_b_01ec64.pth"),
    size="b"
)
model = github.MobileSAM(
    checkpoint_path=Path("my_datasets/models/mobile_sam.pt"),
)
```
By default, `device` is equal to `"cuda"` (PyTorch) or `"/GPU:0"` (TensorFlow) for most models.

If you would like to run the model on your CPU, please add `device="cpu"` (PyTorch) or `device="/CPU:0"` (TensorFlow) in the arguments in the cell below.

In [ ]:
model = pytorch.MaskRCNNv2()

### Select a Pixano format dataset

If you haven't already, please refer to the [dataset notebooks](../datasets/) for information on how to import your dataset to Pixano format.

In [ ]:
library_dir = Path("my_datasets/")
dataset_dir = library_dir / "coco_instances"

views = ["image"]
splits = []

### Generate inferences

You have two options when generating inferences.
- With `process_type="pre_ann"`, the annotations will show up in the Pixano app as pre-annotations that you can individually accept as Ground Truth or reject.
- With `process_type="model_run"`, the annotations will instead be displayed in "Model run", a separate category that you can use to visualize the model's predictions and compare them to existing Ground Truth.

In [ ]:
help(model.process_dataset)

In [ ]:
dataset = model.process_dataset(
    dataset_dir=dataset_dir,
    process_type="model_run",
    views=views,
    splits=splits,
    batch_size=1,
    threshold=0.1,
)

## 3. Browsing the dataset

With the generation complete, you can now browse your dataset with the Pixano app and compare your inferences with ground truths.

You can stop the app by restarting the notebook.

In [ ]:
app = App(library_dir)

In [ ]:
app.display()